In [ ]:
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics import silhouette_score
from sklearn.ensemble import RandomForestClassifier
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd

def Hierachical_clustering(input):
    # input = n * m
    # output = n * (m + 1)
    # 測試用Input
    # X = np.random.randint(5, size = (100,2))
    silhouette_avg = []
    for i in range(2, 11):
        model = AgglomerativeClustering(n_clusters=i)
        silhouette_avg.append(silhouette_score(input, model.fit(input).labels_))
    # 用silhouette_avg決定clustering數量
    # plt.plot(range(2,len(input)), silhouette_avg)
    k = np.argmax(silhouette_avg)
    model = AgglomerativeClustering(n_clusters=k)
    model.fit(input)
    labels = model.labels_
    labels = np.expand_dims(labels, 1)
    # 把input和label合併，變成n * (m + 1)輸出
    output = np.concatenate((input, labels),1)

    return output

def random_forest(X_train, y_train):
    # input = n * (m + 1), n * m
    # output = score of y_train classification
    # X_train size = n * (m + 1)，其中前2行為Train, 最後1行為Label
    model = RandomForestClassifier(n_estimators=100, criterion = 'gini')
    label = X_train[:,-1:]
    label = np.squeeze(label)
    model.fit(X_train[:,0:-1], label)
    predicted = model.predict(y_train)
    return predicted

# Test Data
# df_csv = pd.read_csv("final_feats.csv", index_col=0)
# # print(df_csv)
# print(df_csv.shape)
# y = Hierachical_clustering(df_csv)
# print(y.shape)
# pred = random_forest(y, z)
# print(pred)

In [ ]:
import pandas as pd
mRNA_df_std = pd.read_csv("/Users/oo/Desktop/1208/Standardization/StandardScaler/mRNA_StandardScaler_standardization.csv", index_col=0)
miRNA_df_std = pd.read_csv("/Users/oo/Desktop/1208/Standardization/StandardScaler/miRNA_StandardScaler_standardization.csv", index_col=0)
DNAm_df_std = pd.read_csv("/Users/oo/Desktop/1208/Standardization/StandardScaler/DNAm_StandardScaler_standardization.csv", index_col=0)
CNV_df_std = pd.read_csv("/Users/oo/Desktop/1208/Standardization/StandardScaler/CNV_StandardScaler_standardization.csv", index_col=0)

In [ ]:
mRNA_test = pd.read_csv("/Users/oo/Desktop/1208/independent testing data/final_mRNA_df.csv", index_col=0)
miRNA_test = pd.read_csv("/Users/oo/Desktop/1208/independent testing data/final_miRNA_df.csv", index_col=0)
DNAm_test = pd.read_csv("/Users/oo/Desktop/1208/independent testing data/final_DNAm_df.csv", index_col=0)
CNV_test = pd.read_csv("/Users/oo/Desktop/1208/independent testing data/CNV_df.csv", index_col=0)

In [ ]:
miRNA_df = pd.read_csv("/Users/oo/Desktop/1208/validation data/miRNA.csv", index_col=0)
miRNA_omics = miRNA_df.iloc[:, 4:]
miRNA_omics.columns = miRNA_omics.columns.str.lower()
miRNA_test.columns = miRNA_test.columns.str.lower()
miRNA_used_cols = miRNA_omics.columns[miRNA_omics.columns.str.lower().isin(miRNA_test.columns.str.lower())]

In [ ]:
miRNA_X_train = miRNA_omics[miRNA_used_cols]
miRNA_X_test = miRNA_test[miRNA_used_cols]
miRNA_X_train.to_csv("/Users/oo/Desktop/1208/no selection/miRNA_X_train.csv")
miRNA_X_test.to_csv("/Users/oo/Desktop/1208/no selection/miRNA_X_test.csv")

In [ ]:
DNAm_df = pd.read_csv("/Users/oo/Desktop/1208/validation data/DNAm.csv")
DNAm_df_omics = DNAm_df.iloc[:, 5:]
DNAm_uesd_cols = DNAm_df_omics.columns[DNAm_df_omics.columns.isin(DNAm_test.columns)]
DNAm_X_train = DNAm_df_omics[DNAm_uesd_cols]
DNAm_X_test = DNAm_test[DNAm_uesd_cols]
DNAm_X_train.to_csv("/Users/oo/Desktop/1208/no selection/DNAm_X_train.csv")
DNAm_X_test.to_csv("/Users/oo/Desktop/1208/no selection/DNAm_X_test.csv")

In [ ]:
from final import random_forest
import pandas as pd
CNV_uesd_cols = pd.read_csv("/Users/oo/Desktop/1208/cnv_test.csv", index_col=0)
CNV_uesd_genes = CNV_uesd_cols['gene'].values
y_train = pd.read_csv("/Users/oo/Desktop/1208/validation data/miRNA.csv", usecols=['label'])
CNV_omics = pd.read_csv("/Users/oo/Desktop/1208/no selection/CNV_X_train.csv", index_col=0)
CNV_X_test = pd.read_csv("/Users/oo/Desktop/1208/no selection/CNV_X_test.csv", index_col=0)
CNV_omics = CNV_omics[CNV_uesd_genes]
CNV_X_test = CNV_X_test[CNV_uesd_genes]
labels = random_forest(CNV_omics, y_train, CNV_X_test)

In [ ]:
CNV_X_test['label'] = labels
clinical_df = CNV_test.iloc[:, 2:4]
CNV_X_test_final = pd.concat([clinical_df, CNV_X_test], axis=1)
CNV_X_test_final.reset_index(drop=True)

# CNV_X_test_final.to_csv("/Users/oo/Desktop/1208/cnv_test.csv")

In [ ]:
from lifelines import KaplanMeierFitter
from lifelines.utils import concordance_index
from lifelines.statistics import logrank_test
from lifelines import CoxPHFitter
def cal_c_index(omics_X_test):

    # kmf1 = KaplanMeierFitter()
    # kmf2 = KaplanMeierFitter()
    # cph1 = CoxPHFitter(penalizer=0.1)
    # cph2 = CoxPHFitter(penalizer=0.1)



    g1 = omics_X_test[omics_X_test['label'] == 0]
    g2 = omics_X_test[omics_X_test['label'] == 1]



    duration_1 = g1['duration']
    duration_2 = g2['duration']

    event_1 = g1['vital_status']
    event_2 = g2['vital_status']

    # cph1.fit(g1.drop(columns='label'), event_col='vital_status', duration_col='duration')
    # cph2.fit(g2.drop(columns='label'), event_col='vital_status', duration_col='duration')


    # kmf1.fit(duration_1, event_1)
    # kmf2.fit(duration_2, event_2)

    results = logrank_test(duration_1, duration_2, event_observed_A=event_1, event_observed_B=event_2)
    p_value = results.p_value

    # c_index_1 = concordance_index(duration_1, -cph1.predict_partial_hazard(g1), event_1)
    # c_index_2 = concordance_index(duration_2, -cph2.predict_partial_hazard(g2), event_2)
    # c_index = (c_index_1 + c_index_2) / 2
    print(f'log-rank P-value: {p_value}')
    # print(f'C-index: {c_index}\nC-index1: {c_index_1}\nC-index2: {c_index_2}')
    return p_value
# c_index, p_value = cal_c_index(CNV_X_test_final)


In [ ]:
import matplotlib.pyplot as plt
from lifelines import KaplanMeierFitter
def plot_kmf(omics_X_test, p_value):
    # omics_X_test with label, duration, vital_status

    kmf = KaplanMeierFitter()
    # plt.figure(figsize=(12, 8))
    for i in range(2):
        cluster_data = omics_X_test[omics_X_test['label'] == i]
        
        kmf.fit(cluster_data['duration'], event_observed=cluster_data['vital_status'], label=f'G{i+1}(n={cluster_data.shape[0]})')
        kmf.plot_survival_function(ci_show=False)
        

    plt.figtext(0.6, 0.5, f"log-rank P-value: {p_value:.2e}\n")
    plt.title('Kaplan-Meier Curves by Cluster')
    plt.xlabel('Time')
    # plt.xlim([0, 2696])
    plt.ylabel('Survival Probability')
    plt.show()
# plot_kmf(CNV_X_test_final)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
import warnings
warnings.filterwarnings('ignore')
def random_forest(X_train, y_train, X_test):
    # input = n * (m + 1), n * m
    # output = score of y_train classification
    # X_train size = n * (m + 1)，其中前2行為Train, 最後1行為Label
    model = RandomForestClassifier(n_estimators=100, criterion='gini')
    model.fit(X_train, y_train)
    predicted = model.predict(X_test)
    return predicted
from sklearn.preprocessing import StandardScaler
import pandas as pd
ss = StandardScaler()
miRNA_X_train = pd.read_csv("/Users/oo/Desktop/1208/no selection/miRNA_X_train.csv", index_col=0)
miRNA_X_test = pd.read_csv("/Users/oo/Desktop/1208/no selection/miRNA_X_test.csv", index_col=0)
y_train = pd.read_csv("/Users/oo/Desktop/1208/validation data/miRNA.csv", usecols=['label'])
ss.fit(miRNA_X_train)
miRNA_X_train_ss = ss.transform(miRNA_X_train)
miRNA_X_test_ss = ss.transform(miRNA_X_test)
miRNA_labels = random_forest(miRNA_X_train, y_train, miRNA_X_test)
miRNA_test = pd.read_csv("/Users/oo/Desktop/1208/independent testing data/final_miRNA_df.csv", index_col=0)
miRNA_test_clinical = miRNA_test.iloc[:, 1:3]
miRNA_X_test_ss = pd.DataFrame(miRNA_X_test_ss)
miRNA_X_test_ss['label'] = miRNA_labels
miRNA_test_final = pd.concat([miRNA_test_clinical, miRNA_X_test_ss], axis=1)



In [ ]:
from sklearn.ensemble import RandomForestClassifier
import warnings
warnings.filterwarnings('ignore')
def random_forest(X_train, y_train, X_test):
    # input = n * (m + 1), n * m
    # output = score of y_train classification
    # X_train size = n * (m + 1)，其中前2行為Train, 最後1行為Label
    model = RandomForestClassifier(n_estimators=100, criterion='gini')
    model.fit(X_train, y_train)
    predicted = model.predict(X_test)
    return predicted
from sklearn.preprocessing import StandardScaler
import pandas as pd
while(True):
    plt.clf()
    plt.cla()
    ss = StandardScaler()
    # mRNA_X_train = pd.read_csv("/Users/oo/Desktop/1208/no selection/mRNA_X_train.csv", index_col=0)
    # mRNA_X_test = pd.read_csv("/Users/oo/Desktop/1208/no selection/mRNA_X_test.csv", index_col=0)
    # y_train = pd.read_csv("/Users/oo/Desktop/1208/validation data/miRNA.csv", usecols=['label'])
    ss.fit(mRNA_X_train)
    mRNA_X_train_ss = ss.transform(mRNA_X_train)
    mRNA_X_test_ss = ss.transform(mRNA_X_test)
    mRNA_labels = random_forest(mRNA_X_train_ss, y_train, mRNA_X_test_ss)
    mRNA_test = pd.read_csv("/Users/oo/Desktop/1208/independent testing data/final_mRNA_df.csv", index_col=0)
    mRNA_test_clinical = mRNA_test.iloc[:, 1:3]
    mRNA_X_test_ss = pd.DataFrame(mRNA_X_test_ss)
    mRNA_X_test_ss['label'] = mRNA_labels
    mRNA_test_final = pd.concat([mRNA_test_clinical, mRNA_X_test_ss], axis=1)
    plot_kmf(mRNA_test_final, cal_c_index(mRNA_test_final))
    if cal_c_index(mRNA_test_final) < 0.05:
        break



In [ ]:
from sklearn.ensemble import RandomForestClassifier
import warnings
warnings.filterwarnings('ignore')
def random_forest(X_train, y_train, X_test):
    # input = n * (m + 1), n * m
    # output = score of y_train classification
    # X_train size = n * (m + 1)，其中前2行為Train, 最後1行為Label
    model = RandomForestClassifier(n_estimators=100, criterion='gini')
    model.fit(X_train, y_train)
    predicted = model.predict(X_test)
    return predicted
from sklearn.preprocessing import StandardScaler
import pandas as pd
ss = StandardScaler()
CNV_X_train = pd.read_csv("/Users/oo/Desktop/1208/no selection/CNV_X_train.csv", index_col=0)
CNV_X_test = pd.read_csv("/Users/oo/Desktop/1208/no selection/CNV_X_test.csv", index_col=0)

# CNV_uesd_cols = pd.read_csv("/Users/oo/Desktop/1208/cnv_test.csv", index_col=0)
# CNV_uesd_genes = CNV_uesd_cols['gene'].values
# CNV_X_train = CNV_X_train[CNV_uesd_genes]
# CNV_X_test = CNV_X_test[CNV_uesd_genes]
y_train = pd.read_csv("/Users/oo/Desktop/1208/validation data/CNV.csv", usecols=['label'])
ss.fit(CNV_X_train)
CNV_X_train_ss = ss.transform(CNV_X_train)
CNV_X_test_ss = ss.transform(CNV_X_test)
CNV_labels = random_forest(CNV_X_train, y_train, CNV_X_test)
CNV_test = pd.read_csv("/Users/oo/Desktop/1208/independent testing data/CNV_df.csv", index_col=0)
CNV_test_clinical = CNV_test.iloc[:, 2:4]
CNV_X_test_ss = pd.DataFrame(CNV_X_test)
CNV_X_test_ss['label'] = CNV_labels
CNV_test_final = pd.concat([CNV_test_clinical.reset_index(drop=True), CNV_X_test_ss.reset_index(drop=True)], axis=1)
plot_kmf(CNV_test_final, cal_c_index(CNV_test_final))

In [ ]:
from sklearn.ensemble import RandomForestClassifier
import warnings
warnings.filterwarnings('ignore')
def random_forest(X_train, y_train, X_test):
    # input = n * (m + 1), n * m
    # output = score of y_train classification
    # X_train size = n * (m + 1)，其中前2行為Train, 最後1行為Label
    model = RandomForestClassifier(n_estimators=100, criterion='gini')
    model.fit(X_train, y_train)
    predicted = model.predict(X_test)
    return predicted
from sklearn.preprocessing import StandardScaler
import pandas as pd

ss = StandardScaler()
DNAm_X_train = pd.read_csv("/Users/oo/Desktop/1208/no selection/DNAm_X_train.csv", index_col=0)
DNAm_X_test = pd.read_csv("/Users/oo/Desktop/1208/no selection/DNAm_X_test.csv", index_col=0)

y_train = pd.read_csv("/Users/oo/Desktop/1208/validation data/DNAm.csv", usecols=['label'])
ss.fit(DNAm_X_train)
DNAm_X_train_ss = ss.transform(DNAm_X_train)
DNAm_X_test_ss = ss.transform(DNAm_X_test)
DNAm_labels = random_forest(DNAm_X_train, y_train, DNAm_X_test)
DNAm_test = pd.read_csv("/Users/oo/Desktop/1208/independent testing data/final_DNAm_df.csv", index_col=0)
DNAm_test_clinical = DNAm_test.iloc[:, 1:3]
DNAm_X_test_ss = pd.DataFrame(DNAm_X_test)
DNAm_X_test_ss['label'] = DNAm_labels
DNAm_test_final = pd.concat([DNAm_test_clinical.reset_index(drop=True), DNAm_X_test_ss.reset_index(drop=True)], axis=1)
plot_kmf(DNAm_test_final, cal_c_index(DNAm_test_final))